Data Augmentation


In [ ]:
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
from tqdm import tqdm
import pickle
import IPython.display as ipd  # To play sound in the notebook

In [ ]:
%cd '/content/drive/MyDrive/kaggle'

/content/drive/MyDrive/kaggle


In [ ]:
ref = pd.read_csv("/content/drive/MyDrive/kaggle/Data_path.csv")
ref.head()

,labels,source,path
0,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a01.wav
1,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a13.wav
2,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a08.wav
3,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a07.wav
4,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a05.wav


In [ ]:
def noise(data):
    """
    Adding White Noise.
    """
    noise_amp = 0.05*np.random.uniform()*np.amax(data)   # more noise reduce the value to 0.5
    data = data.astype('float64') + noise_amp * np.random.normal(size=data.shape[0])
    return data
    
def shift(data):
    """
    Random Shifting.
    """
    s_range = int(np.random.uniform(low=-5, high = 5)*1000)  #default at 500
    return np.roll(data, s_range)

def speedNpitch(data):
    """
    peed and Pitch Tuning.
    """
    # you can change low and high here
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.2  / length_change # try changing 1.0 to 2.0 ... =D
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

####################################
# the confusion matrix heat map plot
####################################
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



In [ ]:
# Note this takes a couple of minutes (~16 mins) as we're iterating over 4 datasets, and with augmentation  
df_noise = pd.DataFrame(columns=['feature'])
df_speedpitch = pd.DataFrame(columns=['feature'])
df_shift = pd.DataFrame(columns=['feature'])
df_stretch = pd.DataFrame(columns=['feature'])
df_dynchange = pd.DataFrame(columns=['feature'])

cnt = 0


# loop feature extraction over the entire dataset
for i in tqdm(ref.path):
    
    # first load the audio 
    X, sample_rate = librosa.load(i
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )



    # random shifting (omit for now)
    # Stretch
    # pitch (omit for now)
    # dyn change
    
    # noise 
    aug1 = noise(X)
    aug1 = np.mean(librosa.feature.mfcc(y=aug1, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_noise.loc[cnt] = [aug1]

    # speed pitch
    aug2 = speedNpitch(X)
    aug2 = np.mean(librosa.feature.mfcc(y=aug2, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_speedpitch.loc[cnt] = [aug2]   

    #shift
    aug3 = shift(X)
    aug3 = np.mean(librosa.feature.mfcc(y=aug3, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_shift.loc[cnt] = [aug3]   

    #stretch
    aug4 = stretch(X, rate=0.8)
    aug4 = np.mean(librosa.feature.mfcc(y=aug4, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_stretch.loc[cnt] = [aug4] 

    #dyn_change
    aug5 = dyn_change(X)
    aug5 = np.mean(librosa.feature.mfcc(y=aug5, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_dynchange.loc[cnt] = [aug5] 


    cnt += 1

df_noise.head()

100%|██████████| 12162/12162 [2:23:47<00:00,  1.41it/s]


,feature
0,"[-5.972158930580647, -3.6670977097132345, -2.4..."
1,"[-8.481334077597857, -10.483600332251484, -11...."
2,"[-8.412230112615138, -8.535267688516068, -11.7..."
3,"[-16.868836537292186, -18.22328210648961, -16...."
4,"[-12.278699274216581, -11.151359710958056, -10..."


In [ ]:
df_noise

,labels,source,path,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
0,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a01.wav,-11.113934,-7.215755,-6.219190,-5.926544,-5.850419,-4.808961,-2.513002,-3.500537,-3.721044,-4.487559,-5.429011,-7.305573,-7.141931,-6.555575,-4.516299,-2.556396,-2.251576,-5.133349,-5.566184,-7.205380,-7.909172,-10.862606,-13.266886,-13.715888,-15.576243,-8.864893,-5.869705,-3.969150,-4.066767,-6.243663,-5.814454,-7.329674,-8.715132,-8.864925,-10.491897,-11.772941,-12.314169,...,-12.589432,-10.942276,-8.624939,-7.904713,-7.413645,-7.423730,-7.509189,-8.355754,-7.710210,-6.842867,-6.921706,-5.891709,-6.173982,-7.203384,-6.565605,-6.319575,-7.027235,-5.030657,-3.495362,-1.834732,-0.264224,-1.404713,-0.967948,-1.117432,-0.404290,-0.242711,-0.302327,0.449721,-0.479031,0.026320,-0.603852,-1.322739,-2.316860,-2.422245,-2.315499,-2.479946,-2.781229,-2.827133,0.000211,4.071346
1,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a13.wav,-23.959705,-23.641237,-23.807817,-23.633392,-24.053619,-23.991434,-22.732632,-23.319134,-23.543303,-20.106087,-12.931273,-9.987322,-8.590012,-8.474195,-7.384241,-7.268951,-4.534677,-4.703911,-7.749865,-7.613797,-9.760238,-11.486393,-15.556131,-16.588884,-16.038898,-15.789227,-11.143799,-6.868958,-5.548477,-6.429990,-9.095602,-12.125213,-17.555153,-22.985527,-24.984636,-24.412731,-18.205723,...,1.476247,2.301728,1.738366,0.819691,-0.160614,-0.633167,-1.752202,-0.538234,0.227238,0.536303,0.122309,-0.118428,-0.048525,-0.159905,0.190754,-0.783526,-2.835584,-2.010585,-0.932638,-0.278571,-1.162867,-1.439095,-1.110169,0.081408,0.928278,1.600851,1.365779,1.242357,1.911020,2.144672,1.773134,1.861501,1.698031,1.830862,1.409923,1.540002,0.646922,-1.969410,-4.359469,-5.843833
2,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a08.wav,-27.551817,-25.248302,-22.912870,-23.200481,-24.561079,-24.916075,-24.402773,-23.608076,-24.458662,-26.309509,-17.455606,-12.566965,-9.477596,-8.501862,-6.108100,-5.471074,-6.081676,-7.459237,-13.114928,-15.257000,-12.997003,-13.713450,-18.609718,-21.843933,-22.382586,-23.097433,-24.328823,-23.972639,-14.147175,-11.284182,-10.598524,-10.438836,-11.025839,-11.915486,-7.537542,-4.022674,-3.923109,...,-5.961816,-4.794624,-3.905344,-4.493817,-5.621204,-6.094255,-7.378241,-10.514814,-14.388071,-14.723815,-16.503757,-16.777874,-17.194626,-16.139874,-17.144262,-19.614722,-21.451578,-20.868764,-20.436643,-19.482571,-17.345562,-18.148776,-18.670175,-17.278164,-18.761699,-19.483225,-19.202345,-17.846291,-17.498497,-18.069747,-19.005939,-19.509696,-20.864048,-21.340830,-20.905073,-18.945785,-19.789533,-19.603415,-18.862935,-20.331016
3,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a07.wav,-25.975302,-22.825100,-21.225157,-20.821096,-19.913891,-20.578136,-22.024477,-20.963707,-14.915606,-11.356565,-8.028171,-7.824315,-6.978572,-6.419868,-6.952181,-6.290453,-6.222239,-6.046099,-6.252188,-6.546420,-7.132365,-7.785093,-6.405703,-5.995083,-6.307068,-7.279632,-6.432416,-6.740263,-6.658838,-6.845708,-5.724951,-5.513878,-4.800431,-6.113788,-5.261055,-6.077095,-5.832385,...,-17.465073,-17.236946,-18.353404,-20.352863,-21.428191,-19.903049,-17.890631,-18.803811,-20.066781,-19.211148,-19.321916,-18.702062,-18.851887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a05.wav,-22.458635,-22.467833,-25.884357,-27.827045,-27.593534,-26.666508,-18.659025,-11.698080,-10.721752,-11.915209,-6.850739,-3.500977,-4.898458,-9.883922,-11.935939,-12.674129,-13.808365,-13.950251,-13.646174,-13.029181,-12.490106,-12.303979,-12.577011,-11.460180,-10.485011,-9.496677,-8.059871,-8.585896,-11.165752,-11.631902,-10.897665,-10.621311,-10.252358,-10.343521,-9.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/kaggle/Data_path_features.csv")

In [ ]:
df.shape

(12162, 219)

In [ ]:
df_noise = pd.concat([ref,pd.DataFrame(df_noise['feature'].values.tolist())],axis=1)
df_speedpitch = pd.concat([ref,pd.DataFrame(df_speedpitch['feature'].values.tolist())],axis=1)
df_shift = pd.concat([ref,pd.DataFrame(df_shift['feature'].values.tolist())],axis=1)
df_stretch = pd.concat([ref,pd.DataFrame(df_stretch['feature'].values.tolist())],axis=1)
df_dynchange =pd.concat([ref,pd.DataFrame(df_dynchange['feature'].values.tolist())],axis=1)

print(df_noise.shape,df_speedpitch.shape,df_shift.shape,df_stretch.shape,df_dynchange.shape)

(12162, 219) (12162, 219) (12162, 219) (12162, 273) (12162, 219)


In [ ]:
df_aug = pd.concat([df,df_noise,df_speedpitch,df_shift,df_stretch,df_dynchange],axis=0,sort=False)


In [ ]:
aug_df == 0).all()

,labels,source,path,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269
0,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a01.wav,-11.113934,-7.215755,-6.219190,-5.926544,-5.850419,-4.808961,-2.513002,-3.500537,-3.721044,-4.487559,-5.429011,-7.305573,-7.141931,-6.555575,-4.516299,-2.556396,-2.251576,-5.133349,-5.566184,-7.205380,-7.909172,-10.862606,-13.266886,-13.715888,-15.576243,-8.864893,-5.869705,-3.969150,-4.066767,-6.243663,-5.814454,-7.329674,-8.715132,-8.864925,-10.491897,-11.772941,-12.314169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a13.wav,-23.959705,-23.641237,-23.807817,-23.633392,-24.053619,-23.991434,-22.732632,-23.319134,-23.543303,-20.106087,-12.931273,-9.987322,-8.590012,-8.474195,-7.384241,-7.268951,-4.534677,-4.703911,-7.749865,-7.613797,-9.760238,-11.486393,-15.556131,-16.588884,-16.038898,-15.789227,-11.143799,-6.868958,-5.548477,-6.429990,-9.095602,-12.125213,-17.555153,-22.985527,-24.984636,-24.412731,-18.205723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a08.wav,-27.551817,-25.248302,-22.912870,-23.200481,-24.561079,-24.916075,-24.402773,-23.608076,-24.458662,-26.309509,-17.455606,-12.566965,-9.477596,-8.501862,-6.108100,-5.471074,-6.081676,-7.459237,-13.114928,-15.257000,-12.997003,-13.713450,-18.609718,-21.843933,-22.382586,-23.097433,-24.328823,-23.972639,-14.147175,-11.284182,-10.598524,-10.438836,-11.025839,-11.915486,-7.537542,-4.022674,-3.923109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a07.wav,-25.975302,-22.825100,-21.225157,-20.821096,-19.913891,-20.578136,-22.024477,-20.963707,-14.915606,-11.356565,-8.028171,-7.824315,-6.978572,-6.419868,-6.952181,-6.290453,-6.222239,-6.046099,-6.252188,-6.546420,-7.132365,-7.785093,-6.405703,-5.995083,-6.307068,-7.279632,-6.432416,-6.740263,-6.658838,-6.845708,-5.724951,-5.513878,-4.800431,-6.113788,-5.261055,-6.077095,-5.832385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,male_angry,SAVEE,/content/drive/MyDrive/kaggle/ALL/DC_a05.wav,-22.458635,-22.467833,-25.884357,-27.827045,-27.593534,-26.666508,-18.659025,-11.698080,-10.721752,-11.915209,-6.850739,-3.500977,-4.898458,-9.883922,-11.935939,-12.674129,-13.808365,-13.950251,-13.646174,-13.029181,-12.490106,-12.303979,-12.577011,-11.460180,-10.485011,-9.496677,-8.059871,-8.585896,-11.165752,-11.631902,-10.897665,-10.621311,-10.252358,-10.343521,-9.781135,-8.000402,-6.027526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12157,female_disgust,CREMA,/content/drive/MyDrive/kaggle/AudioWAV/1091_WS...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [ ]:
aug_df=df_aug.fillna(0)


In [ ]:
(aug_df == 0).all()

labels    False
source    False
path      False
0         False
1         False
          ...  
265       False
266       False
267       False
268       False
269       False
Length: 489, dtype: bool

In [ ]:
aug_df =pd.read_csv("Data_aug_features.csv")

In [ ]:
aug_df.shape

(72972, 489)

In [ ]:
# Split between train and test 
X_train, X_test, y_train, y_test = train_test_split(aug_df.drop(['path','labels','source'],axis=1)
                                                    , aug_df.labels
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=42
                                                   )

# Lets see how the data present itself before normalisation 
X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269
54939,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,-60.777382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58844,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59519,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25764,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3094,-20.916286,-20.87188,-21.604078,-21.815535,-22.637621,-23.333347,-23.891783,-26.493162,-26.159685,-24.60396,-24.807512,-26.400805,-25.826202,-25.278959,-24.548157,-23.974998,-24.858067,-25.717644,-27.124403,-27.067141,-27.126019,-29.279362,-28.044325,-26.093863,-26.276482,-26.675251,-27.098228,-27.251453,-26.492544,-27.268381,-27.529369,-26.49843,-26.936363,-30.28232,-28.927889,-30.079018,-31.603024,-31.080626,-32.500095,-33.276554,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29438,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39822,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000

In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Check the dataset now 
X_train[150:160]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269
54939,0.361723,0.369566,0.383936,0.383421,0.382697,0.382429,0.381918,0.381392,0.381042,0.381009,0.380907,0.380651,0.380008,0.379764,0.379315,0.378905,0.378475,0.378427,0.378082,0.378135,0.377957,0.377872,0.377614,0.377635,0.377299,0.377224,0.377016,0.376990,0.376516,0.376463,0.376193,0.376215,0.376270,0.376702,0.376870,0.377395,0.378088,0.378878,0.379607,0.381036,...,-3.989319,-3.991393,-3.994033,-3.997999,-4.098470,-4.101908,-4.106110,-4.109861,-4.208106,-4.211909,-4.211909,0.240315,0.24019,0.240063,0.240063,0.2351,0.235014,0.234977,0.231554,0.231466,0.231466,0.231341,0.226525,0.226347,0.226303,0.226258,0.221228,0.221145,0.221107,0.216918,0.216872,0.216781,0.216739,0.212052,0.212052,0.212012,0.20786,0.207814,0.207723,0.207633
58844,0.361723,0.369566,0.383936,0.383421,0.382697,0.382429,0.381918,0.381392,0.381042,0.381009,0.380907,0.380651,0.380008,0.379764,0.379315,0.378905,0.378475,0.378427,0.378082,0.378135,0.377957,0.377872,0.377614,0.377635,0.377299,0.377224,0.377016,0.376990,0.376516,0.376463,0.376193,0.376215,0.376270,0.376702,0.376870,0.377395,0.378088,0.378878,0.379607,0.381036,...,0.259561,0.259405,0.259245,0.258967,0.252262,0.252057,0.251775,0.251529,0.245494,0.245286,0.245286,0.240315,0.24019,0.240063,0.240063,0.2351,0.235014,0.234977,0.231554,0.231466,0.231466,0.231341,0.226525,0.226347,0.226303,0.226258,0.221228,0.221145,0.221107,0.216918,0.216872,0.216781,0.216739,0.212052,0.212052,0.212012,0.20786,0.207814,0.207723,0.207633
59519,0.361723,0.369566,0.383936,0.383421,0.382697,0.382429,0.381918,0.381392,0.381042,0.381009,0.380907,0.380651,0.380008,0.379764,0.379315,0.378905,0.378475,0.378427,0.378082,0.378135,0.377957,0.377872,0.377614,0.377635,0.377299,0.377224,0.377016,0.376990,0.376516,0.376463,0.376193,0.376215,0.376270,0.376702,0.376870,0.377395,0.378088,0.378878,0.379607,0.381036,...,0.259561,0.259405,0.259245,0.258967,0.252262,0.252057,0.251775,0.251529,0.245494,0.245286,0.245286,0.240315,0.24019,0.240063,0.240063,0.2351,0.235014,0.234977,0.231554,0.231466,0.231466,0.231341,0.226525,0.226347,0.226303,0.226258,0.221228,0.221145,0.221107,0.216918,0.216872,0.216781,0.216739,0.212052,0.212052,0.212012,0.20786,0.207814,0.207723,0.207633
25764,0.361723,0.369566,0.383936,0.383421,0.382697,0.382429,0.381918,0.381392,0.381042,0.381009,0.380907,0.380651,0.380008,0.379764,0.379315,0.378905,0.378475,0.378427,0.378082,0.378135,0.377957,0.377872,0.377614,0.377635,0.377299,0.377224,0.377016,0.376990,0.376516,0.376463,0.376193,0.376215,0.376270,0.376702,0.376870,0.377395,0.378088,0.378878,0.379607,0.381036,...,0.259561,0.259405,0.259245,0.258967,0.252262,0.252057,0.251775,0.251529,0.245494,0.245286,0.245286,0.240315,0.24019,0.240063,0.240063,0.2351,0.235014,0.234977,0.231554,0.231466,0.231466,0.231341,0.226525,0.226347,0.226303,0.226258,0.221228,0.221145,0.221107,0.216918,0.216872,0.216781,0.216739,0.212052,0.212052,0.212012,0.20786,0.207814,0.207723,0.207633
3094,-1.773649,-1.754301,-1.660698,-1.685282,-1.768532,-1.838789,-1.896793,-2.149660,-2.124790,-1.978488,-2.002683,-2.159964,-2.110524,-2.063595,-1.999367,-1.947979,-2.039556,-2.126037,-2.267777,-2.265057,-2.274018,-2.486225,-2.369380,-2.180848,-2.201622,-2.242452,-2.286648,-2.302489,-2.232299,-2.309832,-2.339811,-2.240419,-2.291645,-2.629584,-2.504031,-2.628311,-2.795174,-2.758757,-2.923114,-3.021290,...,0.259561,0.259405,0.259245,0.258967,0.252262,0.252057,0.251775,0.251529,0.245494,0.245286,0.245286,0.240315,0.24019,0.240063,0.240063,0.2351,0.235014,0.234977,0.231554,0.231466,0.231466,0.231341,0.226525,0.226347,0.226303,0.226258,0.221228,0.221145,0.221107,0.216918,0.216872,0.216781,0.216739,0.212052,0.212052,0.212012,0.20786,0.207814,0.207723,0.207633
29438,0.361723,0.369566,0.383936,0.383421,0.3

In [ ]:
# Lets few preparation steps to get it into the correct format for Keras 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)

# Pickel the lb object for future use 
filename = 'labels'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()

(54729, 486)
['female_angry' 'female_disgust' 'female_fear' 'female_happy'
 'female_neutral' 'female_sad' 'female_surprise' 'male_angry'
 'male_disgust' 'male_fear' 'male_happy' 'male_neutral' 'male_sad'
 'male_surprise']


In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

(54729, 486, 1)

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
# New model
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(14)) # Target class number
model.add(Activation('softmax'))
opt = optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 486, 256)          2304      
_________________________________________________________________
activation (Activation)      (None, 486, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 486, 256)          524544    
_________________________________________________________________
batch_normalization (BatchNo (None, 486, 256)          1024      
_________________________________________________________________
activation_1 (Activation)    (None, 486, 256)          0         
_________________________________________________________________
dropout (Dropout)            (None, 486, 256)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 60, 256)           0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(X_train, y_train, batch_size=16, epochs=100, validation_data=(X_test, y_test),verbose=2)

Epoch 1/100
3421/3421 - 133s - loss: 2.1044 - accuracy: 0.2722 - val_loss: 2.1664 - val_accuracy: 0.2758
Epoch 2/100
3421/3421 - 138s - loss: 2.0273 - accuracy: 0.2945 - val_loss: 2.1116 - val_accuracy: 0.2821
Epoch 3/100
3421/3421 - 131s - loss: 1.9705 - accuracy: 0.3149 - val_loss: 2.0575 - val_accuracy: 0.3098
Epoch 4/100


KeyboardInterrupt: ignored